
## Resumen ejecutivo

Este proyecto trata de **predecir las ventas diarias (unit\_sales) de cada producto en cada tienda** de la cadena minorista “Corporación Favorita” para **optimizar inventarios**, **evitar desabastecimientos y sobrestock**, y **mejorar la planificación de promociones**. La variable objetivo es **unit\_sales** y la métrica de evaluación es la **Normalized Weighted Root Mean Squared Logarithmic Error (NWRMSLE)**, la cual penaliza de forma logarítmica los errores y pondera más los productos perecibles ([Kaggle][1]). Los datos provienen de un histórico de ventas diarias (2012–2017) extraídos del sistema POS de Favorita y puestos a disposición en Kaggle, junto con información complementaria de transacciones, precios del petróleo, feriados y metadatos de tiendas y productos ([GitHub][2], [Kaggle][3]).

---

## 1. ¿Cuál es el problema de negocio o la pregunta de investigación?

**Problema de negocio:** Prever la demanda diaria de cada combinación (tienda, producto) para minimizar costos de **stock‑out** (pérdida de ventas) y **overstock** (costos de almacenamiento y caducidad) en la cadena minorista ([GitHub][4]).
**Pregunta de investigación:** ¿Cómo varían las ventas unitarias por tienda y producto día a día, y cómo pueden optimizarse los niveles de inventario y las promociones basándose en esas predicciones? ([GitHub][2])
**Métrica a predecir / explicar:**

* **unit\_sales**: número de unidades vendidas (puede ser entero o decimal, p. ej. kg de queso).
* **Métrica de evaluación:** NWRMSLE (Normalized Weighted Root Mean Squared Logarithmic Error), que penaliza logarítmicamente las discrepancias y aplica un peso mayor a productos perecibles (1.25 vs. 1.0) ([Kaggle][1]).

---

## 2. ¿De dónde vienen estos datos?

1. **Origen**:

   * Datos proporcionados por **Corporación Favorita**, cadena de supermercados de Ecuador, y publicados en formato de competencia en **Kaggle** ([Kaggle][3], [Kaggle][1]).
2. **Frecuencia de actualización**:

   * **Histórico de ventas diarias** desde 2012 hasta 2017; el conjunto de **entrenamiento** abarca hasta el 31 de agosto de 2017, y el de **prueba** corresponde al periodo 16 al 31 de agosto de 2017 ([GitHub][2]).
   * Datos adicionales (transacciones, precios de crudo, feriados) también se registran **diariamente**.
3. **Responsable de la generación**:

   * La **división de análisis de datos** de Corporación Favorita recoge la información desde su sistema POS.
   * Kaggle actúa como repositorio y moderador de la competencia, pero no actualiza el dataset: es un **snapshot estático**.

---

## 3. ¿Qué significan cada una de las columnas?

A continuación el **diccionario de datos** de los principales archivos disponibles:

### 3.1 `train.csv`

| Columna                                                            | Tipo     | Unidad                | Dominio esperado / Valores permitidos                                                                  |
| ------------------------------------------------------------------ | -------- | --------------------- | ------------------------------------------------------------------------------------------------------ |
| `id`                                                               | entero   | —                     | Identificador único de fila (solo etiqueta).                                                           |
| `date`                                                             | fecha    | YYYY‑MM‑DD            | Fecha de la venta.                                                                                     |
| `store_nbr`                                                        | entero   | —                     | Código de tienda (1–54).                                                                               |
| `item_nbr`                                                         | entero   | —                     | Código de producto (1–4400 aproximado).                                                                |
| `unit_sales`                                                       | float    | unidades (o kg, etc.) | Ventas diarias. ≥ 0 para ventas, < 0 indica devoluciones.                                              |
| `onpromotion`                                                      | booleano | —                     | Indica si el producto estaba en promoción (True/False). Aprox. 16 % de NaN (se interpreta como False). |
| *Cita: descripción detallada de campos en train.csv* ([Medium][5]) |          |                       |                                                                                                        |

### 3.2 `test.csv`

| Columna                                      | Tipo     | Descripción                                |
| -------------------------------------------- | -------- | ------------------------------------------ |
| `id`                                         | entero   | Identificador único de fila de prueba.     |
| `date`                                       | fecha    | Fechas 2017‑08‑16 a 2017‑08‑31 a predecir. |
| `store_nbr`                                  | entero   | Código de tienda.                          |
| `item_nbr`                                   | entero   | Código de producto.                        |
| `onpromotion`                                | booleano | Indica promoción en el periodo de prueba.  |
| *Cita: estructura de test.csv* ([Medium][5]) |          |                                            |

### 3.3 `stores.csv`

| Columna                                    | Tipo   | Descripción                                              |
| ------------------------------------------ | ------ | -------------------------------------------------------- |
| `store_nbr`                                | entero | Código de tienda.                                        |
| `city`                                     | texto  | Ciudad donde está ubicada la tienda.                     |
| `state`                                    | texto  | Provincia/estado.                                        |
| `type`                                     | texto  | Tipo de tienda (A, B, C, … según tamaño o formato).      |
| `cluster`                                  | entero | Agrupación de tiendas similares (análisis de clústeres). |
| *Cita: metadatos de tiendas* ([Medium][6]) |        |                                                          |

### 3.4 `items.csv`

| Columna                                      | Tipo   | Descripción                                                      |
| -------------------------------------------- | ------ | ---------------------------------------------------------------- |
| `item_nbr`                                   | entero | Código de producto.                                              |
| `family`                                     | texto  | Categoría general (e.g., “BEVERAGES”, “DAIRY”).                  |
| `class`                                      | entero | Subcategoría o clase específica dentro de la familia.            |
| `perishable`                                 | entero | 1 si es perecible (peso 1.25 en la métrica), 0 si no (peso 1.0). |
| *Cita: metadatos de productos* ([Medium][6]) |        |                                                                  |

### 3.5 `transactions.csv`

| Columna                                         | Tipo   | Descripción                                         |
| ----------------------------------------------- | ------ | --------------------------------------------------- |
| `date`                                          | fecha  | Fecha de la transacción.                            |
| `store_nbr`                                     | entero | Código de tienda.                                   |
| `transactions`                                  | entero | Número total de transacciones diarias en la tienda. |
| *Cita: detalles de transacciones* ([Medium][6]) |        |                                                     |

### 3.6 `oil.csv`

| Columna                                   | Tipo  | Descripción                                                                                |
| ----------------------------------------- | ----- | ------------------------------------------------------------------------------------------ |
| `date`                                    | fecha | Fecha de referencia.                                                                       |
| `dcoilwtico`                              | float | Precio diario del crudo West Texas Intermediate (WTI), puede influir en costos logísticos. |
| *Cita: precio del petróleo* ([Medium][6]) |       |                                                                                            |

### 3.7 `holidays_events.csv`

| Columna                                  | Tipo  | Descripción                                                            |
| ---------------------------------------- | ----- | ---------------------------------------------------------------------- |
| `date`                                   | fecha | Fecha del feriado o evento.                                            |
| `type`                                   | texto | “Holiday”, “Event”, “Transfer”, etc.                                   |
| `locale`                                 | texto | Ámbito (“National”, “Local”).                                          |
| `locale_name`                            | texto | Nombre de la ciudad o región si es local.                              |
| `description`                            | texto | Nombre descriptivo del feriado/evento.                                 |
| `transferred`                            | bool  | True si la fecha fue trasladada (p. ej., feriado que se pasa a lunes). |
| *Cita: feriados y eventos* ([Medium][6]) |       |                                                                        |

---

[1]: https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/overview/evaluation?utm_source=chatgpt.com "Corporación Favorita Grocery Sales Forecasting - Kaggle"
[2]: https://github.com/danberlyne/favorita-sales-forecasting?utm_source=chatgpt.com "danberlyne/favorita-sales-forecasting - GitHub"
[3]: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data?utm_source=chatgpt.com "Corporación Favorita Grocery Sales Forecasting | Kaggle"
[4]: https://github.com/oberoiheman/Corporaci-n-Favorita-Grocery-Sales-Forecasting?utm_source=chatgpt.com "oberoiheman/Corporaci-n-Favorita-Grocery-Sales-Forecasting"
[5]: https://kishan-122.medium.com/kaggle-corporaci%C3%B3n-favorita-grocery-sales-forecasting-b36e0caa0a9f?utm_source=chatgpt.com "Kaggle Corporación Favorita Grocery Sales Forecasting"
[6]: https://medium.com/%40baakwadegraftmiah/time-series-forecasting-predictive-model-building-for-store-sales-at-favorita-grocery-retailer-b5ef9d262b7b?utm_source=chatgpt.com "Time Series Forecasting & Predictive Model Building for Store Sales ..."
